# 📚 PINNs: Neural Network as Solver of the PDE Solution

## 🔍 Neural Network as Solver of the PDE Solution

We set up a neural network model $u_\theta(x,t)$ to approximate the solution $u(x,t) $ of the PDE.  

The key idea is to **learn $u_\theta$ by minimizing the residuals of the PDE and the boundary/initial conditions**, *without* needing the explicit analytical solution of $ u(x, t)$.


This means our training objective is to find the network parameters $\theta $ such that:

- The PDE residual $ \mathcal{R}(x,t) $ is minimized at **collocation points**,
- The network output respects the **initial condition** 
- The network output respects the **boundary conditions** 


To achieve this, we **pass the residual loss functions** we defined earlier:

- $ \mathcal{L}_{\text{PDE}} $ for the PDE residual,
- $ \mathcal{L}_{\text{IC}} $ for the initial condition,
- $ \mathcal{L}_{\text{BC}} $ for the boundary condition,

to an optimizer that minimizes their weighted sum (**the deep learning training process**):

$ \min_{\theta} \ \mathcal{L}(\theta) = \mathcal{L}_{\text{PDE}} + \lambda_{\text{IC}} \mathcal{L}_{\text{IC}} + \lambda_{\text{BC}} \mathcal{L}_{\text{BC}}$

This defines the optimization problem underlying our Physics-Informed Neural Network (PINN) framework.


### Imports and setup

In [ ]:
!pip install git+https://github.com/mikelunizar/T4-PINNs.git

In [ ]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from pinn.src.pde_solver import PDESolver
from pinn.src.visuals import plot_collocation_setup, visualize_pde_error_domain
from pinn.src.residuals import evaluate_residual_on_collocation_points
import warnings


# Reproducibility
np.random.seed(53)
pl.seed_everything(53)

# Device configuration
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## 🌐 Define the PDE Domain and Sampling Strategy

In this section, we define the **PDE domain** where our Physics-Informed Neural Network (PINN) will operate.  
We discretize the space-time domain, sample **collocation points** where the PDE will be enforced, and define **boundary** and **initial conditions**.





#### 🧮 Domain $ (x,t) $

We construct a uniform grid in the $(x, t)$ domain:  
- $x \in [-1, 1]$  
- $t \in [0, 1]$  
- Resolution: `Nx` points in space and `Nt` points in time  

This results in a 2D grid of size $ \text{Nx} \times \text{Nt} $.


In [ ]:
Nx, Nt = 300, 200
# set grid PDE domain at resolution Nx, Nt
x = np.linspace(-1, 1, Nx)
t = np.linspace(0, 1, Nt)
X, T = np.meshgrid(x, t)
xt_grid = np.stack([X.ravel(), T.ravel()], axis=-1)  # shape (N, 2) where N = Nx * Nt

#### 🔍 Collocation Points

From the full grid, we randomly sample:  
- `Ncp` **collocation points** where the PDE will be enforced  
- These are split into training and validation sets (e.g., 80% / 20%)

In [ ]:
def collocation_points_generation(xt_grid, Ncp, val_ratio=0.2):
    # Sample Ncp from grid where to evaluate PDE
    xt_cp = xt_grid[np.random.choice(len(xt_grid), Ncp, replace=False)]
    # Split the data train, val (80-20%)
    train_xt_cp, valid_xt_cp, _, _ = train_test_split(xt_cp, np.zeros(len(xt_cp)), test_size=val_ratio, random_state=52)

    return train_xt_cp, valid_xt_cp

In [ ]:
# Generate collocation points
Ncp = 500 # Number of Collocation points
coll_points_train, coll_points_val = collocation_points_generation(xt_grid, Ncp, val_ratio=0.2)

#### 🧱 Boundary Conditions (BC)

We select `Nbc` points along the **spatial boundaries** $x = -1$ and $x = 1$, across different times.  
We impose homogeneous Dirichlet boundary conditions:

- $u(-1, t) = u(1, t) = 0$


In [ ]:
def boundary_condition_points(Nbc, xt_grid):
    # Define Boundary Conditions
    xt_bc = xt_grid[np.argwhere((xt_grid[:, 0] == 1) | (xt_grid[:, 0] == -1)).reshape(-1)]  # all points at x=-1 and x=1
    xt_bc = xt_bc[np.random.choice(len(xt_bc), Nbc, replace=False)]
    u_bc = np.zeros(xt_bc.shape[0]).reshape(-1, 1)  # set bc of u(-1, t) = u(1, t) = 0
    
    return xt_bc, u_bc

In [ ]:
# Generate boundary condition points (x,t) and solution u(x,t)
Nbc = 100  # Number of Boundary condition points
xt_bc, u_bc = boundary_condition_points(Nbc, xt_grid)

#### 🧨 Initial Conditions (IC)

We select `Nic` points at the **initial time** $t = 0$, across the entire spatial domain.  
The initial condition is given by:

- $u(x, 0) = -\sin(\pi x)$


In [ ]:
def initial_condition_points(Nic, xt_grid):
    # Define Initial  Conditions
    xt_ic = xt_grid[np.argwhere(xt_grid[:, 1] == 0).reshape(-1)]  # All points with t=0
    xt_ic = xt_ic[np.random.choice(len(xt_ic), Nic, replace=False)]
    u_ic = -np.sin(np.pi*xt_ic[:, 0]).reshape(-1, 1)  # set ic u(x, 0) = - sin(pi*x) 

    return xt_ic, u_ic

In [ ]:
# Generate initial condition points (x,t) and solution u(x,t)
Nic = 100  # Number Initial condition points
xt_ic, u_ic = initial_condition_points(Nic, xt_grid)

#### 📊 Visualization

The figure below shows:  
- The full space-time  
- Collocation points (training and validation)  
- Boundary condition points (colored by $u$)  
- Initial condition points (colored by $u$)

This setup forms the foundation for training the PINN to solve the PDE.

In [ ]:
plot_collocation_setup(xt_grid, coll_points_train, coll_points_val, xt_bc, u_bc, xt_ic, u_ic)

## 🧩 PINN Loss Functions: Residuals and Conditions

Our goal is to train a neural network $ u(x,t) $ that satisfies the PDE:

##### $ u_t + u \cdot u_x - \nu u_{xx} = 0 $

where:
- $ u_t = \frac{\partial u}{\partial t} $ (time derivative)  
- $ u_x = \frac{\partial u}{\partial x} $ (spatial derivative)  
- $ u_{xx} = \frac{\partial^2 u}{\partial x^2} $ (second spatial derivative)  
- $ \nu=0.025 $ is the viscosity coefficient.



#### PDE Residual Loss

At **collocation points** $ (x,t) $, the PDE residual should be zero:

$ \mathcal{R}(x,t) = u_t + u u_x - \nu u_{xx} \approx 0 $

The PDE loss minimizes the mean squared residual:

$ \mathcal{L}_{\text{PDE}} = \frac{1}{N_{cp}} \sum_{i=1}^{N_{cp}} \mathcal{R}(x_i, t_i)^2 $


In [ ]:
# Physical parameters
nu = 0.025 

def residual_pde(xt_cp, model):
    """
    Compute PDE residual loss at collocation points:
    """
    x = xt_cp[:, 0:1].clone().detach().requires_grad_(True)  # activate gradients
    t = xt_cp[:, 1:].clone().detach().requires_grad_(True)   # activate gradients
    
    with torch.enable_grad():
        inputs = torch.cat([x, t], dim=1)
        u_hat = model(inputs)
        
        # First derivatives
        du_dt = torch.autograd.grad(u_hat, t, grad_outputs=torch.ones_like(u_hat), create_graph=True)[0]
        du_dx = torch.autograd.grad(u_hat, x, grad_outputs=torch.ones_like(u_hat), create_graph=True)[0]
        
        # Second derivative wrt x
        d2u_dx2 = torch.autograd.grad(du_dx, x, grad_outputs=torch.ones_like(du_dx), create_graph=True)[0]

        # PDE residual: u_t + u * u_x - nu * u_xx = 0
        residual = du_dt + u_hat * du_dx - nu * d2u_dx2
        
        residual_pde_ = torch.mean(residual ** 2)
        
    return residual_pde_


#### Initial Condition Residual Loss

At initial time $ t=0 $, the network output must match the initial condition:

$ u(x,0) = u_{\text{ic}}(x) $

The initial condition loss works as a data-driven approach, because the solution $u(x,t)$ is known:

$ \mathcal{L}_{\text{IC}} = \frac{1}{N_{ic}} \sum_{i=1}^{N_{ic}} \left( u(x_i,0) - u_{\text{ic}}(x_i) \right)^2 $


In [ ]:
def residual_ic(xt_ic, u_ic, model):
    """
    Compute MSE loss for Initial Condition:
    """
    u_pred = model(xt_ic)
    return torch.mean((u_ic - u_pred) ** 2)

#### Boundary Condition Residual Loss

On spatial boundaries $ x = \pm 1 $, enforce:

$ u(-1, t) = u(1, t) = 0 $

The boundary loss is:

$ \mathcal{L}_{\text{BC}} = \frac{1}{N_{bc}} \sum_{i=1}^{N_{bc}} \left( u(x_i, t_i) - u_{\text{bc}}(x_i, t_i) \right)^2 $

In [ ]:
def residual_bc(xt_bc, u_bc, model):
    """
    Compute MSE loss for Boundary Condition:
    """
    u_pred = model(xt_bc)
    return torch.mean((u_pred - u_bc) ** 2)

#### Total Residual to minimize

The total loss is typically a weighted sum:

$
\mathcal{L} = \mathcal{L}_{\text{PDE}} + \lambda_{\text{IC}} \mathcal{L}_{\text{IC}} + \lambda_{\text{BC}} \mathcal{L}_{\text{BC}}
$

which is minimized during training to solve the PDE.

In [ ]:
lambda_ic, lambda_bc = 1, 1  # set weights

def residual_loss(xt_cp, xt_bc, u_bc, xt_ic, u_ic, model):
    # PDE residual at collocation points
    residual_pde_cost = residual_pde(xt_cp, model)
    # Initial Conditions residual at i.c. points
    residual_ic_cost = residual_ic(xt_ic, u_ic, model)
    # Boundary Condition residual at b.c. points
    residual_bc_cost = residual_bc(xt_bc, u_bc, model)

    # Total residual: Weighted sum
    residual = residual_pde_cost + lambda_ic * residual_ic_cost +  lambda_bc * residual_bc_cost

    return {'residual': residual, 'residual_pde': residual_pde_cost, 'residual_bc': residual_bc_cost, 'residual_ic': residual_ic_cost} 

## Neural Network Architecture

The model is a fully connected feedforward neural network defined as:

$
\text{Input} \rightarrow \text{Linear} \rightarrow \text{Activation} \rightarrow \text{Linear} \rightarrow \text{Activation}\rightarrow \text{Linear} \rightarrow (Output)
$

- **Input dimension**: 2 (raw $(x,t)$ or 22 if Fourier feature encoding is used.
- **Hidden layers**: 3 layers, each with 64 neurons.
- **Activation**: Hyperbolic tangent $\tanh$ applied after each hidden layer.


#### Choose the Activation Function

Activation functions affect how well the network learns and how smooth the predicted solution is.

**Think about:**  
- What happens if the activation is not smooth or not differentiable everywhere?  
- How might that impact the gradients needed for training a PINN?  
- Why could having multiple continuous derivatives be important for solving PDEs?

Choose wisely and test how it influences your results!

| Activation | Continuity       | Differentiability              | Non-zero Derivatives                    |
|------------|------------------|-------------------------------|------------------------------------|
| **ReLU**   | Continuous       | Not differentiable at 0        | 1st derivative non-zero except at 0 | 
| **Tanh**   | Continuous       | Infinitely differentiable       | All derivatives are non-zero        |
| **Sigmoid**| Continuous       | Infinitely differentiable       | All derivatives are non-zero        | 

In [ ]:
# input, output and hidden size
input_size = 2
ouput_size = 1
hiden_size = 64 

# NN model
model = torch.nn.Sequential(torch.nn.Linear(input_size, hiden_size),
                            torch.nn.Tanh(),
                            torch.nn.Linear(hiden_size, hiden_size),
                            torch.nn.Tanh(),
                            torch.nn.Linear(hiden_size, hiden_size),
                            torch.nn.Tanh(),
                            torch.nn.Linear(hiden_size, ouput_size))

## Solving the PDE with PINNs: Training the Model

Now that you have designed your neural network and chosen the activation functions, it’s time to **train the model** to solve the PDE using a Physics-Informed Neural Network (PINN).

During training, the network learns to minimize the PDE residual while satisfying the boundary and initial conditions by adjusting its parameters.


In [ ]:
import warnings; warnings.simplefilter('ignore')

# hyperparameters
opt = torch.optim.Adam
lr = 1e-3
bs = 16
epochs = 100
foldername = f'bs={bs}_lr={lr}_Ncp={Ncp}' # Run name

# Solver setup
pde_solver = PDESolver(model, residual=residual_loss,
               xt_bc=xt_bc, u_bc=u_bc, xt_ic=xt_ic, u_ic=u_ic,
               lr=lr, optimizer=opt, device=device)

# Callbacks
early_stopping = pl.callbacks.EarlyStopping(monitor='valid_residual', patience=25, mode='min') # regularization technique
logger = WandbLogger(name=foldername, project='T4-PINNs') # W&B logger

# Trainer setup
trainer = pl.Trainer(accelerator='gpu' if torch.cuda.is_available() else 'cpu',
                     max_epochs=epochs, 
                     logger=logger,callbacks=[early_stopping],
                     check_val_every_n_epoch=10)

# dataloaders
train_loader = DataLoader(torch.tensor(coll_points_train, dtype=torch.float32), batch_size=32, shuffle=True)
valid_loader = DataLoader(torch.tensor(coll_points_val, dtype=torch.float32), batch_size=32, shuffle=False)

# Training the model
trainer.fit(pde_solver, train_loader, valid_loader)

### Inference: Provide Dense Solution

One of the coolest advantages of PINNs is that, once trained, the neural network acts as a **continuous function approximator** over the whole domain.

This means we can perform **inference** to predict the solution at **any point** inside the domain—not just at training points or a fixed grid.

**PINNs**:

- We get a **dense, smooth solution** without needing to solve the PDE again on a fine mesh.
- This flexibility allows us to:
  - Visualize the solution in high resolution,
  - Evaluate the solution at points of interest,
  - Use the model for further analysis or control tasks.
  
In essence, the PINN provides a **mesh-free, continuous representation** of the PDE solution, making it incredibly versatile compared to classical numerical methods.

---


In [ ]:
# perform prediction
u_grid_hat = model(torch.tensor(xt_grid, dtype=torch.float32)).detach()

# Create a figure with 1 row and 2 columns of subplots
fig, (ax1) = plt.subplots(1, 1)
# Left subplot - prediction
sc1 = ax1.scatter(xt_grid[:, 0:1], xt_grid[:, 1:], c=u_grid_hat, cmap='jet')
ax1.set_title(f'Resolution {Nx}x{Nt}')
fig.colorbar(sc1, ax=ax1)
plt.show()

In [ ]:
# High resolution Solution
Nx_high_res, Nt_high_res = 1000, 1000
# set grid PDE domain at resolution Nx, Nt
x_high_res = np.linspace(-1, 1, Nx_high_res)
t_high_res = np.linspace(0, 1, Nt_high_res)
X_high_res, T_high_res = np.meshgrid(x_high_res, t_high_res)
xt_high_res = np.stack([X_high_res.ravel(), T_high_res.ravel()], axis=-1)  # shape (N, 2) where N = Nx * Nt
# perform prediction
u_grid_hihg_res_hat = model(torch.tensor(xt_high_res, dtype=torch.float32)).detach()

# Create a figure with 1 row and 2 columns of subplots
fig, (ax1) = plt.subplots(1, 1)
# Left subplot - prediction
sc1 = ax1.scatter(xt_high_res[:, 0:1], xt_high_res[:, 1:], c=u_grid_hihg_res_hat, cmap='jet')
ax1.set_title(f'Resolution {Nx_high_res}x{Nt_high_res}')
fig.colorbar(sc1, ax=ax1)
plt.show()

### PDE Residual over Domain

When solving PDEs with Physics-Informed Neural Networks (PINNs), **plotting the PDE residual over the domain is crucial**.

**Why?**

- The **residual** measures how well the network's predicted solution satisfies the underlying PDE at each point in the domain.
- Since PINNs incorporate the PDE directly into the loss function, a low residual means the solution closely follows the physics.
- Importantly, the residual can be computed **even where we don't have the exact or reference solution available**.

This makes the PDE residual a powerful, model-based **error indicator**:

- It allows us to **estimate the local error** of the solution without requiring ground truth data.
- We can identify regions in the domain where the model struggles, guiding refinement strategies such as:
  - Adding more collocation points,
  - Increasing network capacity,
  - Adjusting training parameters.

**In summary:**  
Visualizing the residual helps us **trust** and **diagnose** the PINN solution beyond known points, giving insight into the quality and reliability of the PDE solver in unexplored areas.


In [ ]:
# Compute the residuals
residuals = evaluate_residual_on_collocation_points(xt_grid, model, residual_pde_fn=residual_pde)
residuals = residuals.reshape(T.shape)  # if xt_grid was created from meshgrid (X, T)

In [ ]:
# Visualize the residual errors
visualize_pde_error_domain(residuals, X, T, coll_points_train, coll_points_val, log=False)
visualize_pde_error_domain(residuals, X, T, coll_points_train, coll_points_val, log=True)

### PINN Solution vs. Finite Differences

After obtaining the PINN solution, it is important to compare it with a high-quality (HQ) reference solution computed using classical numerical methods, such as Finite Differences (FD).

**Key point:**  
The PINN was trained **only using a relatively small set of collocation points**, often far fewer than traditional numerical methods require.  
In contrast, the Finite Differences solution is computed on a **fine mesh with over one million nodes ($> 10^6$)**, providing a very detailed and accurate reference.

This comparison showcases how PINNs can approximate complex PDE solutions efficiently, using far fewer data points than classical mesh-based methods.


In [ ]:
def generate_finite_differences_solution():
    # Target resolution
    x_target = np.linspace(-1, 1, Nx)
    t_target = np.linspace(0, 1, Nt)
    
    # Fine resolution for stability
    Nx_fine = 1024
    x_fine = np.linspace(-1, 1, Nx_fine)
    dx = x_fine[1] - x_fine[0]
    
    dt = 0.00001
    Nt_fine = int(1.0 / dt)
    save_times = np.linspace(0, Nt_fine-1, Nt, dtype=int)  # time indices to save
    
    # Initialize fine grid
    u = -np.sin(np.pi * x_fine)
    u[0] = 0
    u[-1] = 0
    
    # Output array (Nt, Nx)
    u_all = np.zeros((Nt, Nx))
    
    # Save initial state (interpolated)
    u_all[0] = np.interp(x_target, x_fine, u)
    
    frame = 1
    for n in range(1, Nt_fine):
        un = u.copy()
        u[1:-1] = (un[1:-1]
                   - dt * un[1:-1] * (un[2:] - un[:-2]) / (2 * dx)
                   + nu * dt * (un[2:] - 2 * un[1:-1] + un[:-2]) / dx**2)
        u[0] = 0
        u[-1] = 0
    
        if n == save_times[frame]:
            u_all[frame] = np.interp(x_target, x_fine, u)
            frame += 1
            if frame == Nt:
                break
                
    return u_all


def plot_pinn_vs_finite_diff_results(xt_grid, u_grid_hat, u_solution):
    """
    Plot predicted solution, target solution, and absolute error for PINNs.

    Parameters:
    - xt_grid: Tensor or ndarray of shape (N, 2) with (x,t) coordinates.
    - u_grid_hat: Tensor or ndarray of predicted solution values of shape (N,).
    - u_solution: ndarray or tensor of target solution values (can be multidim, will be flattened).
    """
    u_target = torch.tensor(u_solution.flatten(), dtype=torch.float32)
    error = torch.abs(u_grid_hat.flatten() - u_target)

    fig, axs = plt.subplots(1, 3, figsize=(24, 6))

    # Prediction subplot
    sc1 = axs[0].scatter(xt_grid[:, 0], xt_grid[:, 1], c=u_grid_hat.flatten(), cmap='jet', s=5)
    axs[0].set_title('Predicted Solution')
    fig.colorbar(sc1, ax=axs[0])

    # Target subplot
    sc2 = axs[1].scatter(xt_grid[:, 0], xt_grid[:, 1], c=u_target, cmap='jet', s=5)
    axs[1].set_title('Target Solution')
    fig.colorbar(sc2, ax=axs[1])

    # Error subplot
    sc3 = axs[2].scatter(xt_grid[:, 0], xt_grid[:, 1], c=error, cmap='inferno', s=5)
    axs[2].set_title('Absolute Error')
    fig.colorbar(sc3, ax=axs[2])

    for ax in axs:
        ax.set_xlabel('$x$')
        ax.set_ylabel('$t$')

    plt.tight_layout()
    plt.show()

In [ ]:
# Generate Finite Differences Solution
u_solution = generate_finite_differences_solution()

In [ ]:
# Compare solution PINN vs Finite Differences
plot_pinn_vs_finite_diff_results(xt_grid, u_grid_hat, u_solution)

# 🚀 Do It Yourself: Explore New Physics!

Now it’s your turn!  
Let’s explore how changing the **Initial Conditions (IC)** leads to entirely new solutions of the same PDE.

🧠 **Remember:**  
For every combination of **PDE**, **Initial Conditions (IC)**, and **Boundary Conditions (BC)**, there's one **unique solution** (under well-posed conditions).  
That means even small changes to the initial setup can drastically affect the behavior of the solution!

---

### 🧨 Try New Initial Conditions: New Physics, New Patterns!

We define the initial condition at **time $t = 0$** by selecting `Nic` points across the spatial domain.  
This is your opportunity to see how the system evolves from different starting states.

Here are some examples to try:

- $u(x, 0) = -\sin(\pi x)$
- $u(x, 0) = -\sin(2\pi x)$
- $u(x, 0) = -\sin(4\pi x)$
- $u(x, 0) = \text{your custom function!} \; 🎨$

---

Go ahead, tweak the IC, retrain the PINN, and **discover how the physics responds**.  
This is where PINNs shine: solving new problems with just a few modifications and no re-meshing!

🎯 **Challenge:** Try a custom IC and compare the result to a known FD solution.  
Can you get a good match? Where does it fail?

---

> ✨ Tip: Use the residual and error plots to evaluate how your new IC affects training and accuracy.

### 🎚️ Tune the Physics: Adjust Loss Weights

Want more control over how your model learns the physics?  
You can **tweak the weights** of the residual loss components to influence how strongly the model enforces:

- ⚓ **Boundary Conditions (BC)**
- 🕒 **Initial Conditions (IC)**
- 🧮 **PDE Residual**


In [ ]:
def initial_condition_points(Nic, xt_grid):
    # Define Initial  Conditions
    xt_ic = xt_grid[np.argwhere(xt_grid[:, 1] == 0).reshape(-1)]  # All points with t=0
    xt_ic = xt_ic[np.random.choice(len(xt_ic), Nic, replace=False)]
    
    # here is where you define the initial conditions
    u_ic = #TODO  -np.sin(4*np.pi*xt_ic[:, 0]).reshape(-1, 1)  # set ic u(x, 0) = - sin(pi*x) 

    return xt_ic, u_ic

In [ ]:
lambda_ic, lambda_bc = #TODO set weights residuals

def residual_loss(xt_cp, xt_bc, u_bc, xt_ic, u_ic, model):
    # PDE residual at collocation points
    residual_pde_cost = residual_pde(xt_cp, model)
    # Initial Conditions residual at i.c. points
    residual_ic_cost = residual_ic(xt_ic, u_ic, model)
    # Boundary Condition residual at b.c. points
    residual_bc_cost = residual_bc(xt_bc, u_bc, model)

    # Total residual: Weighted sum
    residual = residual_pde_cost + lambda_ic * residual_ic_cost +  lambda_bc * residual_bc_cost

    return {'residual': residual, 'residual_pde': residual_pde_cost, 'residual_bc': residual_bc_cost, 'residual_ic': residual_ic_cost} 

In [ ]:
# Choose the collocation points
Ncp = #TODO 5000
Nic = #TODO 300
Nbc = #TODO 300

# Generate collocation points
coll_points_train, coll_points_val = collocation_points_generation(xt_grid, Ncp, val_ratio=0.2)

# Generate initial condition points (x,t) and solution u(x,t)
xt_ic, u_ic = initial_condition_points(Nic, xt_grid)

# Generate initial condition points (x,t) and solution u(x,t)
xt_bc, u_bc = boundary_condition_points(Nbc, xt_grid)

# Visualize your new problem
plot_collocation_setup(xt_grid, coll_points_train, coll_points_val, xt_bc, u_bc, xt_ic, u_ic)

## 🏗️ Model Architecture & Solve Your New PDE

Now that you've chosen a new **Initial Condition**, it's time to build your own PINN model to solve the updated PDE problem!


🔍 **Reminder:** Smooth activation functions like `tanh` are often better for PDEs, especially when we need to compute higher-order derivatives.


In [ ]:
# input, output and hidden size
input_size = 2
ouput_size = 1
hiden_size = 128 

# NN model
model = torch.nn.Sequential(torch.nn.Linear(input_size, hiden_size),
                            torch.nn.Tanh(),
                            torch.nn.Linear(hiden_size, hiden_size),
                            torch.nn.Tanh(),
                            torch.nn.Linear(hiden_size, hiden_size),
                            torch.nn.Tanh(),
                            torch.nn.Linear(hiden_size, ouput_size))


### 🛠️ 2. Set Your Optimization Hyperparameters




In [ ]:
# hyperparameters
opt = torch.optim.Adam
lr = 1e-3
bs = 16
epochs = 2000
foldername = f'bs={bs}_lr={lr}_Ncp={Ncp}' # Run name

In [ ]:
# Solver setup
pde_solver = PDESolver(model, residual=residual_loss,
               xt_bc=xt_bc, u_bc=u_bc, xt_ic=xt_ic, u_ic=u_ic,
               lr=lr, optimizer=opt, device=device)

# Callbacks
early_stopping = pl.callbacks.EarlyStopping(monitor='valid_residual', patience=25, mode='min') # regularization technique
logger = WandbLogger(name=foldername, project='T4-PINNs') # W&B logger

# Trainer setup
trainer = pl.Trainer(accelerator='gpu' if torch.cuda.is_available() else 'cpu',
                     max_epochs=epochs, 
                     logger=logger,callbacks=[early_stopping],
                     check_val_every_n_epoch=10)

# dataloaders
train_loader = DataLoader(torch.tensor(coll_points_train, dtype=torch.float32), batch_size=32, shuffle=True)
valid_loader = DataLoader(torch.tensor(coll_points_val, dtype=torch.float32), batch_size=32, shuffle=False)

# Training the model
trainer.fit(pde_solver, train_loader, valid_loader)



### Plot PDE High resolution Solution

In [ ]:
# High resolution Solution
Nx_high_res, Nt_high_res = 500, 500
# set grid PDE domain at resolution Nx, Nt
x_high_res = np.linspace(-1, 1, Nx_high_res)
t_high_res = np.linspace(0, 1, Nt_high_res)
X_high_res, T_high_res = np.meshgrid(x_high_res, t_high_res)
xt_high_res = np.stack([X_high_res.ravel(), T_high_res.ravel()], axis=-1)  # shape (N, 2) where N = Nx * Nt
# perform prediction
u_grid_hihg_res_hat = u_pde_model(torch.tensor(xt_high_res, dtype=torch.float32)).detach()

# Create a figure with 1 row and 2 columns of subplots
fig, (ax1) = plt.subplots(1, 1)
# Left subplot - prediction
sc1 = ax1.scatter(xt_high_res[:, 0:1], xt_high_res[:, 1:], c=u_grid_hihg_res_hat, cmap='jet')
ax1.set_title(f'Resolution {Nx_high_res}x{Nt_high_res}')
fig.colorbar(sc1, ax=ax1)
plt.show()